##SAI SRIKANTH SARABU
##CWID: A20343781

In [1]:
!pip install pyspark

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#importing required modules 
from pyspark import SparkContext, SparkConf
import numpy as np
import pandas as pd  
from pyspark.mllib.linalg.distributed import CoordinateMatrix, MatrixEntry
from pyspark import sql
from pyspark.sql import SparkSession,SQLContext

In [4]:
#initializing spark context and spark session
sc = SparkContext("local","Assignment-3: HITS using Spark")
spark = SparkSession.builder.getOrCreate()

In [5]:
data = '/content/drive/MyDrive/Sample Data/email-Eu-core.txt' #datapath

In [6]:
dataRDD = sc.textFile(data) #converting data to RDD

In [7]:
#converting each node to into intergers
allEdges = dataRDD.map(lambda x: x.split()).map(lambda x: (int(x[0]),int(x[1])))

In [8]:
#getting node count
nodeCount = allEdges.flatMap(lambda x:x).distinct().count()
nodeCount

1005

In [9]:
#making all tupples of all the edges with source node, destination node and 1  
edgesRDD = allEdges.map(lambda x: (x[0], x[1], 1))

In [10]:
#using CoordinateMatrix to get adjacency L of edge direction  
L = CoordinateMatrix(edgesRDD,nodeCount,nodeCount).toBlockMatrix()
LT = L.transpose()         #taking transpose of the L

In [11]:
hTupples = []                                             #initialinzing list for initial h value
for i in range(nodeCount):                                #looping node count
  hTupples.append((i, 0, 1))                              #initializing 1 in h tupple
h = CoordinateMatrix(sc.parallelize(hTupples)).toBlockMatrix()        #converting into RDD and initializing h
a = None                                                  #initializing a
λ = 1                                                     #initializing λ
μ = 1                                                     #initializing μ
I = 40
for i in range(I):                                        #iterating 40 times
  aL = LT.multiply(h)                                     #mutlipling 𝐿𝑇*ℎ to compute a
  aScalefactor = []                                       #initializing scale factor array for a
  aMax = np.max(np.array(aL.toLocalMatrix().toArray()))   #getting max element the aL after converting to array
  for i in range(nodeCount):                              
    aScalefactor.append((i, i, 1/aMax))                   #appending tupples to construct matrix with only diagonal elements as 1/(max element from a)
  a = CoordinateMatrix(sc.parallelize(aScalefactor)).toBlockMatrix().multiply(aL) #scaling each value batween 0 and 1 in RDD by mutlipying scalling factor with aL
  hL = L.multiply(a)                                      #mutlipling L*a to compute h
  hScalefactor = []                                       #initializing scale factor array for h
  hMax = np.max(np.array(hL.toLocalMatrix().toArray()))   #getting max element the hL after converting to array
  for j in range(nodeCount):                              
    hScalefactor.append((j, j, 1/hMax))                   #appending tupples to construct matrix with only diagonal elements as 1/(max element from h) 
  h = CoordinateMatrix(sc.parallelize(hScalefactor)).toBlockMatrix().multiply(hL)  #scaling each value batween 0 and 1 in RDD by mutlipying scalling factor with hL

In [12]:
hArray = np.array(h.toLocalMatrix().toArray())        #converting h to numpy array
aArray = np.array(a.toLocalMatrix().toArray())        #converting a to numpy array
hSort = np.argsort(hArray, axis = 0)                  #sorting the array using argsort which returns indices of the sorted values 
hTop5 = hSort[-5:]                                    #getting top 5 hub score
hBot5 = hSort[:5]                                     #getting bottom 5 hub score
print('Top 5 Hub score ')
for i in hTop5:
  print('node number -> ',int(i+1),' Hub Score:',float(hArray[i]))        #adding 1 to i to get the node number
print('\nBottom 5 Hub score')
for i in hBot5:
  print('node number -> ',int(i+1),' Hub Score:',float(hArray[i]))        #Got zeroes due to very less hub score

aSort = np.argsort(aArray, axis = 0)
aTop5 = aSort[-5:]                                    #getting top 5 authority score
aBot5 = aSort[:5]                                     #getting bottom 5 authority score
print('\nTop 5 Authority score ')
for i in aTop5:
  print('node number -> ',int(i+1),' Authority Score:',float(aArray[i]))
print('\nBottom 5 Authority score')
for i in aBot5:
  print('node number -> ',int(i+1),' Authority Score:',float(aArray[i]))    #Got zeroes due to very less authority score


Top 5 Hub score 
node number ->  63  Hub Score: 0.7745555183142752
node number ->  108  Hub Score: 0.8268162873433461
node number ->  122  Hub Score: 0.896653122213392
node number ->  83  Hub Score: 0.9047741513157942
node number ->  161  Hub Score: 1.0

Bottom 5 Hub score
node number ->  1005  Hub Score: 0.0
node number ->  862  Hub Score: 0.0
node number ->  858  Hub Score: 0.0
node number ->  657  Hub Score: 0.0
node number ->  855  Hub Score: 0.0

Top 5 Authority score 
node number ->  122  Authority Score: 0.8962812611093613
node number ->  435  Authority Score: 0.8981523413743511
node number ->  63  Authority Score: 0.9273457680741725
node number ->  108  Authority Score: 0.9553614962609248
node number ->  161  Authority Score: 1.0

Bottom 5 Authority score
node number ->  902  Authority Score: 0.0
node number ->  944  Authority Score: 0.0
node number ->  945  Authority Score: 0.0
node number ->  525  Authority Score: 0.0
node number ->  751  Authority Score: 0.0
